In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc
from scipy.spatial.distance import cosine

In [82]:
data['target'] = train.target

In [85]:
data.corr()

,pair_id,group_id,doc_id,is_train,numbers,vowels,consonants,target
pair_id,1.000000,0.999976,-0.024723,-0.852762,0.028207,0.000656,0.005171,0.041701
group_id,0.999976,1.000000,-0.025279,-0.851937,0.028102,0.000685,0.005197,0.041418
doc_id,-0.024723,-0.025279,1.000000,-0.013971,0.014277,-0.021634,-0.021704,-0.017996
is_train,-0.852762,-0.851937,-0.013971,1.000000,-0.021930,0.000921,-0.003800,NaN
numbers,0.028207,0.028102,0.014277,-0.021930,1.000000,0.272802,0.279844,-0.104665
vowels,0.000656,0.000685,-0.021634,0.000921,0.272802,1.000000,0.974721,0.050869
consonants,0.005171,0.005197,-0.021704,-0.003800,0.279844,0.974721,1.000000,0.052329
target,0.041701,0.041418,-0.017996,NaN,-0.104665,0.050869,0.052329,1.000000


In [ ]:
new_train

In [2]:
train = pd.read_csv('../../data/train_groups.csv')
test  = pd.read_csv('../../data/test_groups.csv')

In [10]:
test['is_train'] = False
train['is_train'] = True


In [11]:
data_x = pd.concat(( train.drop('target', 1), test))

In [5]:
title = pd.read_csv('../../data/docs_titles.tsv', sep='\t', encoding='utf-8', lineterminator='\n')

## Preprocess Text Labels

In [ ]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import re
mystem = Mystem()
russian_stopwords = stopwords.words("russian")
alph = r'[<>\%\(\)abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ/#;:!{}_|\-\?.=&<>@\[\]""]'

information = []
for i in title.title:
    try:
        tokens = mystem.lemmatize(i.lower())
        tokens
        tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
        text = " ".join(tokens)
        text = re.sub(alph, '', text)
        text = re.sub(r'[\s+\d+\n]', ' ', text)
        information.append(text)
        
    except:
        information.append(np.nan)

In [ ]:
title['information'] = information

In [6]:
title = pd.read_csv('nem_title.csv')

In [12]:
data = data_x.merge(title, how = 'left', on = 'doc_id')
data = data.replace(np.nan, 'noinformation')

In [8]:
title = pd.read_csv('nem_title.csv')

# Statistic 

## count of numbers

In [39]:
np.arange(10).astype(str)

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U21')

In [12]:
numbers = []
num = np.arange(10).astype(str)
for i in data.title:
    numbers.append(sum(letter in num for letter in i))

In [13]:
data['numbers'] = numbers

## count of vowel

In [14]:
vowels = []
vowel = ['а', 'о', 'и', 'е', 'ё', 'э', 'ы', 'у', 'ю', 'я']
for i in data.title:
    vowels.append(sum(letter in vowel for letter in i))
data['vowels'] = vowels

## count of consonant

In [15]:
consonants = []
consonant = ['б', 'в', 'г', 'д', 'ж', 'з', 'й', 'к', 'л', 'м', 'н', 'п', 'р','с', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ']
for i in data.title:
    consonants.append(sum(letter in consonant for letter in i))
data['consonants'] = consonants

In [49]:
data.to_csv('data_statistic.csv', index = False)

# Make TfIdf

In [8]:

vectorizer = TfidfVectorizer(max_features=2**10,
                             dtype = 'float32',
                        use_idf=True,
                        ngram_range=(2,2), # considering only 1-grams
) 


tf_idf_text = vectorizer.fit_transform(data.information)

/home/kirill/playground/env/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1547: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  UserWarning)


In [87]:
data_tf_idf[:1]

,автоновость авто,авторский право,администрация город,адрес телефон,адрес час,аквариум зеленеть,аккаунт инстаграм,актер роль,активный отдых,актриса кино,...,cos_14,cos_15,cos_16,cos_17,cos_18,cos_19,cos_20,cos_21,cos_22,cos_23
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.162051e-07,4.921769e-07,5.729361e-07,4.513265e-07,5.256401e-07,6.285633e-07,6.565289e-07,8.339600e-07,9.408054e-07,0.000001


In [13]:
data_tf_idf = pd.DataFrame(tf_idf_text.toarray(), columns = vectorizer.get_feature_names())
data_tf_idf['doc_id'] = data.doc_id
data_tf_idf['pair_id'] = data.pair_id
data_tf_idf['group_id'] = data.group_id
data_tf_idf['is_train'] = data.is_train

In [14]:
data_tf_idf[:2]

,автоновость авто,авторский право,администрация город,адрес телефон,адрес час,аквариум зеленеть,аккаунт инстаграм,актер роль,активный отдых,актриса кино,...,юридический услуга,юрий хованский,язык класс,языковой панель,январь год,ярославский область,doc_id,pair_id,group_id,is_train
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15731,1,1,True
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14829,2,1,True


### SVD 

In [38]:
from sklearn.decomposition import TruncatedSVD
tfidf = TfidfVectorizer(min_df=5, max_features=maxFeats*5, ngram_range=(2,2),
                        strip_accents='unicode',
                        lowercase =True, analyzer='word', token_pattern=r'\w+',
                        use_idf=True, smooth_idf=True, sublinear_tf=True, 
                        stop_words = 'english')

In [40]:

svdT = TruncatedSVD(n_components=800)
svdTFit = svdT.fit_transform(tfidf.fit_transform(data[data.is_train].information))

## Build matrix of distance

In [15]:
data_frame = data['pair_id'].copy().reset_index()

In [19]:
from sklearn.metrics.pairwise import cosine_similarity as cos

In [17]:
col = ['doc_id', 'pair_id', 'group_id', 'is_train']

In [ ]:
cos_matrix = cos(data_tf_idf.drop(col, 1))

In [22]:
svdT = TruncatedSVD(n_components=800)
svdTFit = svdT.fit_transform(cos_matrix)

In [24]:
svd = pd.DataFrame(svdTFit)
svd.to_csv('svdTFit.csv', index = False)

In [ ]:
svdTfit['doc_id']

In [14]:
matrix = pd.DataFrame(cos_matrix)

In [15]:
matrix['is_train'] = data['is_train']

In [ ]:
new_train = matrix[matrix.is_train].drop('is_train', 1)
new_test = matrix[~matrix.is_train].drop('is_train', 1)

In [16]:
data_new = pd.read_csv('data_statistic.csv')

In [18]:
matrix['numbers'] = data_new['numbers']
matrix['vowels'] = data_new['vowels']
matrix['consonants'] = data_new['consonants']

In [ ]:
data1 = pd.read_csv('new_data')

In [24]:
new_train = pd.read_csv('new_train.csv')
new_test = pd.read_csv('new_test.csv')

data1 = pd.concat(( new_train.drop('target', 1), new_test))

In [36]:
matrix['doc_id'] = title['doc_id']

In [ ]:
data1 = data1.merge(matrix, how = 'left', on = 'doc_id')

In [1]:
# matrix['mean_true_hash'] = data1.mean_true_hash
# matrix['cos_mean'] = data1.cos_mean


In [91]:
new_train['numbers'] = data['numbers']
new_train['vowels'] = data['vowels']

In [ ]:
new_test['numbers'] = data['numbers']
new_test['vowels'] = data['vowels']

In [ ]:
matrix.to_csv('cos_matrix.csv', index = False)

## Make Hash

In [6]:
from sklearn.feature_extraction.text import HashingVectorizer


vectorizer = HashingVectorizer(n_features=2**10, dtype = 'float32' ,ngram_range=(1, 3) )
data_all = data_all.replace(np.nan, 'noinformation')
X_text = vectorizer.fit_transform(data.text )
# X = vectorizer.fit_transform(data.information )

In [ ]:
from sklearn.feature_extraction.text import 

In [27]:
len( data_x.set_index('doc_id').iloc[data_x.doc_id.unique()].group_id)

28026

In [25]:
len(data_x.set_index('doc_id').iloc[data_x.doc_id.unique()].group_id)

28026

In [41]:
data_x.doc_id

0    15731
1    14829
2    15764
3    17669
4    14852
Name: doc_id, dtype: int64

In [33]:
data_text.drop(['group_id', 'is_train'], 1, inplace = True)

In [26]:
data_x.shape

(28317, 4)

In [25]:
data_tf_idf.shape

(28317, 16384)

In [27]:
# # data_tf_idf = pd.DataFrame(tf_idf_text.toarray())
# # data_text['doc_id'] = data_x.doc_id.unique()
# # data_text['pair_id'] = data_x.pair_id.unique()
# data_text['group_id'] = data_x.set_index('doc_id').iloc[data_x.doc_id.unique()].sort_values('pair_id').group_id.values
# data_text['is_train'] = data_x.set_index('doc_id').iloc[data_x.doc_id.unique()].sort_values('pair_id').is_train.values

# data_text['target'] = train

In [8]:
data_text.to_csv('text_hash.csv', index = False)

In [4]:
data_text = pd.read_csv('text_hash.csv', dtype = 'float32')

In [46]:
data_text[:10]


,0,1,2,3,4,5,6,7,8,9,...,16378,16379,16380,16381,16382,16383,doc_id,pair_id,group_id,is_train
0,0.000000,0.000000,0.000000,-0.004800,0.000000,0.004800,0.000000,0.000000,0.000000,-0.048002,...,0.000000,0.000000,0.000000,0.004800,0.000000,0.000000,15731,1.0,1,True
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14829,2.0,1,True
2,0.000000,0.002271,0.006814,-0.002271,-0.001136,0.000000,-0.001136,0.010220,0.000000,-0.005678,...,-0.002271,0.000000,0.007949,-0.003407,-0.003407,-0.011356,15764,3.0,1,True
3,-0.002012,-0.000671,0.002012,0.000671,-0.001341,-0.002682,0.000671,0.004694,0.000671,0.000000,...,0.001341,0.001341,0.001341,-0.000671,0.000671,0.002682,17669,4.0,1,True
4,0.000000,-0.004375,0.004375,0.000000,0.000000,0.000000,-0.004375,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.008751,0.000000,14852,5.0,1,True
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029142,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15458,6.0,1,True
6,0.005060,0.000000,-0.005060,0.010121,0.000000,-0.005060,0.000000,0.040483,0.000000,-0.010121,...,0.000000,0.000000,-0.005060,0.000000,0.005060,-0.005060,14899,7.0,1,True
7,0.000000,0.003983,0.000000,0.000000,0.000000,-0.019915,0.000000,0.003983,0.000000,0.003983,...,0.000000,0.000000,-0.003983,0.000000,0.007966,0.003983,16879,8.0,1,True
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.016725,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16310,9.0,1,True
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008515,0.000000,-0.008515,...,0.000000,0.008515,0.000000,0.000000,0.000000,0.000000,15440,10.0,1,True


## Make DataFrame

In [12]:
# data_tfidf.shape
data_hash = pd.DataFrame(X.toarray())
# data_hash.to_csv('data_hash')

In [13]:
data_hash['doc_id'] = data.doc_id
data_hash['pair_id'] = data.pair_id
data_hash['group_id'] = data.group_id
data_hash['is_train'] = data.is_train


### train and test

In [11]:
data_text = data_text.replace(np.nan, 0) 

In [49]:
train_text.to_csv('train_text.csv', index = False)

,0,1,2,3,4,5,6,7,8,9,...,16379,16380,16381,16382,16383,doc_id,pair_id,group_id,is_train,target
0,0.0,0.0,0.0,-0.0048,0.0,0.0048,0.0,0.0,0.0,-0.048002,...,0.0,0.0,0.0048,0.0,0.0,15731,1.0,1,True,0


In [54]:
# train_text = data_text[data_text.is_train]
# train_text['target'] = train.target

test_text= data_text[~data_text.is_train]

### Group by target

In [ ]:
true_vector_mean = new_train.drop(['pair_id', 'is_train','group_id','doc_id'], 1).groupby('target').agg('mean')

In [52]:
true_vector_mean_text = train_text.drop(['pair_id', 'is_train','group_id','doc_id'], 1).groupby('target').agg('mean')

In [53]:
true_vector_mean_text.to_csv('text/true_vector_mean_text.csv', index = False)

In [ ]:
.drop(['pair_id', 'is_train','group_id','doc_id'], 1).groupby('target').agg('mean')

### Group by group_id


In [13]:
train_group_vector_mean = new_train.drop(['pair_id', 'is_train','target'],1).groupby('group_id').agg('mean')

In [50]:
train_group_vector_mean_text = train_text.drop(['pair_id', 'is_train','target'],1).groupby('group_id').agg('mean')

In [51]:
train_group_vector_mean_text.to_csv('text/train_group_vector_mean_text.csv', index = False)

In [26]:
train_group_vector_mean.to_csv('vectors_of_groups.csv', index = False)

### Find the same group in test

In [ ]:
test_group_vector_mean_text = test_text.drop(['pair_id', 'is_train','target'],1).groupby('group_id').agg('mean')

In [ ]:
test_group_vector_mean_text.to_csv('text/test_group_vector_mean_text.csv', index = False)

In [ ]:
test_group_vector_mean = new_test.drop(['pair_id', 'is_train'],1).groupby('group_id').agg('mean')

In [25]:
test_group_vector_mean.to_csv('test_vectors_of_groups.csv', index = False)

### find distanses for mean vectors of groups 

In [ ]:
cos_mean_text = []

for i in tqdm(train_group_vector_mean_text.index):
    q = train_group_vector_mean_text.loc[i]
    for j in train_text[train_text.group_id == i + 1].index:
        cos_mean.append(cosine(q, train_text.drop(['doc_id','pair_id','group_id','is_train','target'],1).loc[j]))
    

In [ ]:
train['cos_mean_text'] = cos_mean
train.head()

In [17]:
cos_mean = []

for i in tqdm(train_vectors_of_groups.index):
    q = train_vectors_of_groups.loc[i]
    for j in new_train[new_train.group_id == i + 1].index:
        cos_mean.append(cosine(q, new_train.drop(['doc_id','pair_id','group_id','is_train','target'],1).loc[j]))
    

  0%|          | 0/129 [00:00<?, ?it/s]/home/kirill/playground/env/lib/python3.5/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 129/129 [54:04<00:00, 23.84s/it]


In [28]:
train1['cos_mean'] = cos_mean
train1.head()

/home/kirill/playground/env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,pair_id,group_id,doc_id,target,is_train,mean_true_hash,train_group,cos_mean
0,1,1,15731,0,True,0.975970,0.0,0.506909
1,2,1,14829,0,True,0.939010,0.0,0.585735
2,3,1,15764,0,True,0.965664,0.0,0.773604
3,4,1,17669,0,True,0.990659,0.0,0.950442
4,5,1,14852,0,True,0.950488,0.0,0.670281


In [30]:
train1.corr()

,pair_id,group_id,doc_id,target,is_train,mean_true_hash,train_group,cos_mean
pair_id,1.000000,0.999900,-0.121821,0.041701,NaN,-0.021072,0.103803,-0.070021
group_id,0.999900,1.000000,-0.122197,0.041418,NaN,-0.020639,0.102131,-0.069258
doc_id,-0.121821,-0.122197,1.000000,-0.017996,NaN,-0.049112,-0.081367,-0.065106
target,0.041701,0.041418,-0.017996,1.000000,NaN,0.142582,0.254589,-0.254728
is_train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean_true_hash,-0.021072,-0.020639,-0.049112,0.142582,NaN,1.000000,0.084109,0.152689
train_group,0.103803,0.102131,-0.081367,0.254589,NaN,0.084109,1.000000,-0.199284
cos_mean,-0.070021,-0.069258,-0.065106,-0.254728,NaN,0.152689,-0.199284,1.000000


In [ ]:
cos_mean_text2 = []
for i in tqdm(test_group_vector_mean_text.index):
    q = test_group_vector_mean_text.loc[i]
    for j in test_text[test_text.group_id == i + 130].index:
        cos_mean2.append(cosine(q, test_text.drop(['doc_id','pair_id','group_id','is_train'],1).loc[j]))

In [ ]:

test['cos_mean_text'] = cos_mean_text2
test.head()

In [19]:
cos_mean2 = []
for i in tqdm(test_vectors_of_groups.index):
    q = test_vectors_of_groups.loc[i]
    for j in new_test[new_test.group_id == i + 130].index:
        cos_mean2.append(cosine(q, new_test.drop(['doc_id','pair_id','group_id','is_train'],1).loc[j]))

  0%|          | 0/180 [00:00<?, ?it/s]/home/kirill/playground/env/lib/python3.5/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 180/180 [1:48:16<00:00, 35.95s/it]


In [20]:

test['cos_mean'] = cos_mean2
test.head()

,pair_id,group_id,doc_id,is_train,cos_mean
0,11691,130,6710,False,0.553029
1,11692,130,4030,False,0.806924
2,11693,130,5561,False,0.641318
3,11694,130,4055,False,0.629449
4,11695,130,4247,False,0.752520


In [70]:
new_train.train_group = new_train.train_group.replace(np.nan, 0)


In [57]:
new_test.train_group = new_test.train_group.replace(np.nan, 0)

In [74]:
new_train.to_csv('new_train.csv', index = False)
new_test.to_csv('new_test.csv', index = False)

In [73]:
new_train.head()

,pair_id,group_id,doc_id,target,is_train,mean_true_hash,train_group
0,1,1,15731,0,True,0.975970,0.0
1,2,1,14829,0,True,0.939010,0.0
2,3,1,15764,0,True,0.965664,0.0
3,4,1,17669,0,True,0.990659,0.0
4,5,1,14852,0,True,0.950488,0.0


## find distances for mean vector of all true data

In [25]:
def make_cos(vector, new_test = new_test):
    cos = []
    for i in  new_test.index:
        cos.append(cosine(vector, new_test.loc[i]))
    return cos

In [35]:
train_mean_true = make_cos(true_false_vector_mean.loc[0], new_test = new_train.drop('target', 1))
test_mean_true = make_cos(true_false_vector_mean.loc[0])


/home/kirill/playground/env/lib/python3.5/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [37]:
train['mean_true_hash'] = train_mean_true
test['mean_true_hash'] = test_mean_true


### Data preporation

In [69]:
doc_to_title = {}
with open('../../data/docs_titles.tsv') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
print (len(doc_to_title))

28026


In [78]:
traingroups_titledata = {}
for i in range(len(train1)):
    new_doc = train1.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title[doc_id]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))
    
testgroups_titledata = {}
for i in range(len(test1)):
    new_doc = test1.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    title = doc_to_title[doc_id]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title))

In [79]:
import numpy as np
y_train = []
X_train = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        y_train.append(target_id)
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train.append(sorted(all_dist, reverse=True)[0:15]    )
X_train = np.array(X_train)
y_train = np.array(y_train)
groups_train = np.array(groups_train)
print (X_train.shape, y_train.shape, groups_train.shape)

(11690, 15) (11690,) (11690,)
